In [1]:
import os
import time
import threading

#### Current Thread

In [5]:
print('Program enter')
print(f'Process id: {os.getpid()}')
print(f'Current Thread name: {threading.current_thread().name}')
threading.current_thread().name = 'My-Thread'
print('Sleeping...')
time.sleep(5)
print(f'Current Thread name: {threading.current_thread().name}')
print('Program exit')

Program enter
Process id: 18311
Current Thread name: MainThread
Sleeping...
Current Thread name: My-Thread
Program exit


#### Creating Threads in Python
- Create Thread class instance and associate a target function with it
- Write a sub-class of Thread, override the run method and instantiate the sub-class

In [9]:
def run():
    for i in range(1,6):
        print(f'{i} - Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
        time.sleep(0.5)

In [20]:
print(f'Main Thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run)
t1.name = 'Thread-01'
t1.start()

print(f't1 is alive: {t1.is_alive()}') # Maybe

t1.join() # Current thread (main thread) goes to wait state, get notified when t1 is dead 

print(f't1 is alive: {t1.is_alive()}') # No

print('Main Thread exit')

Main Thread enter, Process id: 18311
1 - Thread name: Thread-01, Process id: 18311
t1 is alive: True
2 - Thread name: Thread-01, Process id: 18311
3 - Thread name: Thread-01, Process id: 18311
4 - Thread name: Thread-01, Process id: 18311
5 - Thread name: Thread-01, Process id: 18311
t1 is alive: False
Main Thread exit


In [21]:
print(f'Main Thread enter, Process id: {os.getpid()}')

t1 = threading.Thread(target=run)
t1.name = 'Thread-01'

t2 = threading.Thread(target=run)
t2.name = 'Thread-02'

t1.start()
t2.start()

t1.join()
t2.join()

print('Main Thread exit')

Main Thread enter, Process id: 18311
1 - Thread name: Thread-01, Process id: 18311
1 - Thread name: Thread-02, Process id: 18311
2 - Thread name: Thread-01, Process id: 18311
2 - Thread name: Thread-02, Process id: 18311
3 - Thread name: Thread-01, Process id: 18311
3 - Thread name: Thread-02, Process id: 18311
4 - Thread name: Thread-01, Process id: 18311
4 - Thread name: Thread-02, Process id: 18311
5 - Thread name: Thread-01, Process id: 18311
5 - Thread name: Thread-02, Process id: 18311
Main Thread exit


#### Create a Thread whose target function accepts numbers (*args), adds them and display the sum

In [22]:
def add(*args):
    print(f'Thread name: {threading.current_thread().name}, Sum: {sum(args)}')

In [25]:
t1 = threading.Thread(target=add, name='First', args=(1,2,3,4,5))
t1.start()

t2 = threading.Thread(target=add, name='Second', args=(10,20,30))
t2.start()

Thread name: First, Sum: 15
Thread name: Second, Sum: 60


#### Create a Thread by Inheritance
1. Write a sub-class of Thread class
2. Target method by default is the run method, which we override (instance method)
3. t.run is a way to specify target in case we wish to have a different target
4. Arguments can be set explicitly using t.args attribute and accessed as self.args

In [5]:
class MyThread(threading.Thread):
    def do_something(self):
        for i in range(1,6):
            print(f'{i} - Thread name: {threading.current_thread().name}, Process id: {os.getpid()}')
            time.sleep(0.5)

In [8]:
t1 = MyThread()
t1.run = t1.do_something # set the target
t1.name = 'Thread-100'

t2 = MyThread()
t2.run = t2.do_something # set the target
t2.name = 'Thread-200'

t1.start()
t2.start()

1 - Thread name: Thread-100, Process id: 35532
1 - Thread name: Thread-200, Process id: 35532
2 - Thread name: Thread-100, Process id: 35532
2 - Thread name: Thread-200, Process id: 35532
3 - Thread name: Thread-100, Process id: 35532
3 - Thread name: Thread-200, Process id: 35532
4 - Thread name: Thread-100, Process id: 35532
4 - Thread name: Thread-200, Process id: 35532
5 - Thread name: Thread-100, Process id: 35532
5 - Thread name: Thread-200, Process id: 35532


#### Write a sub-class of Thread, name it as AddThread
- the target method sums the thread arguments and display the sum

In [11]:
class AddThread(threading.Thread):
    def run(self):
        print(f'{threading.current_thread().name} --> {sum(self.args)}')

In [13]:
t1 = AddThread()
t1.name = 'MyThread-01'
t1.args = [1,2,3,4,5]

t2 = AddThread()
t2.name = 'MyThread-02'
t2.args = [10,20,30,40,50]

t1.start()
t2.start()

MyThread-01 --> 15
MyThread-02 --> 150


#### Thread assignment
- Write a class named Resource
- It will have an instance field called data, initialized to 0
- It will have an instance method named do_something
- The method will increment data by one and display the name of the current thread followed by current value of data 
- The resource method (do_something) will be the target for a Thread

In [42]:
class Resource:
    
    def __init__(self):
        self.data = 0
    
    def do_something(self):
        self.data += 1
        for i in range(10000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')

In [39]:
class Resource:
    
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
    
    def do_something(self):
        self.lock.acquire()
        self.data += 1
        for i in range(10000000):
            pass
        print(f'{threading.current_thread().name} --> {self.data}')
        self.lock.release()

#### Using lock through 'with' statement
- lock is implicitly acquired and released

In [44]:
class Resource:
    
    def __init__(self):
        self.data = 0
        self.lock = threading.Lock()
    
    def do_something(self):
        with self.lock:
            self.data += 1
            for i in range(10000000):
                pass
            print(f'{threading.current_thread().name} --> {self.data}')

In [35]:
r = Resource() # Shared resource

t1 = threading.Thread(target = r.do_something, name='First')
t2 = threading.Thread(target = r.do_something, name='Second')
t3 = threading.Thread(target = r.do_something, name='Third')

t1.start()
t2.start()
t3.start()

First --> 1
Second --> 2
Third --> 3


In [45]:
r = Resource() # Shared resource

for i in range(10):
    t1 = threading.Thread(target = r.do_something, name=f'Thread-{i+1}')
    t1.start()

Thread-1 --> 1
Thread-2 --> 2
Thread-3 --> 3
Thread-4 --> 4
Thread-5 --> 5
Thread-6 --> 6
Thread-7 --> 7
Thread-8 --> 8
Thread-9 --> 9
Thread-10 --> 10
